In [1]:
#import dependencies
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import difflib


In [5]:

# Load movies data
movies = pd.read_csv('Resources/movies.csv')
#convert all columns except index to string
movies= movies.astype(str)
movies = movies.astype({'index': 'int32'})

In [6]:
movies.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.31295,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [7]:
#selecting features for recommendation
rec_features = ['genres', 'keywords', 'popularity', 'vote_average', 'cast', 'director', 'budget']

In [8]:
#replacing null values

for feature in rec_features:
  movies[feature] = movies[feature].fillna('')

In [9]:
#combine all the selected features
all_features = movies['genres']+' '+movies['keywords']+' '+movies['popularity']+' '+movies['cast']+' '+movies['director']+' '+movies['budget']+' '+movies['vote_average']
all_features

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance nan 0.642552 Edward Burns Kerry...
4800    Comedy Drama Romance TV Movie date love at fir...
4801    nan nan 0.857008 Daniel Henney Eliza Coupe Bil...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [10]:
#Turn the text data into a matrix
vector = TfidfVectorizer(ngram_range =(1,2))
vector_features = vector.fit_transform(all_features)

In [11]:
#get the similarity scores using the cosine similarity
similarity = cosine_similarity(vector_features)

In [12]:
print(similarity)

[[1.         0.01720569 0.01933351 ... 0.         0.         0.        ]
 [0.01720569 1.         0.00487701 ... 0.00581752 0.         0.        ]
 [0.01933351 0.00487701 1.         ... 0.         0.02231449 0.        ]
 ...
 [0.         0.00581752 0.         ... 1.         0.         0.00957057]
 [0.         0.         0.02231449 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.00957057 0.         1.        ]]


In [13]:
#get list of all movie titles
title_list = movies['title'].tolist()


In [21]:

#Ask user for input
movie_title = input('My favorite movie is : ').strip()


In [22]:
# finding the close match for the movie name given by the user
try:
  close_match = difflib.get_close_matches(movie_title, title_list)
  print(close_match)
  closest_match = close_match[0]
  print(closest_match)
except(IndexError):
  print("Movie not in list")

['Monsters, Inc.', 'Monsters', 'Monster']
Monsters, Inc.


In [23]:
#get index of the movie
index_of_movie = movies[movies.title == closest_match]['index'].values[0]
print(index_of_movie)

231


In [24]:
#get similarity scores of all the other movies
similarity_score = list(enumerate(similarity[index_of_movie]))


In [25]:
#sort the scores in descending order
sorted_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

In [26]:
#print recommended movies
i=0
for movie in sorted_movies[1:]:
  index = movie[0]
  suggested_movie = movies[movies.index==index]['title'].values[0]
  if (i<5):
    print(suggested_movie)
    i+=1  

Monsters University
Up
Who Killed the Electric Car?
Inside Out
Monster House
